In [1]:
import numpy as np
import pandas as pd
import os
import gc
import xgboost as xgb
from xgboost import plot_importance
import math
from joblib import Parallel, delayed

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import warnings

warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def get_psi(train,test,f_cols):
    psi_res = pd.DataFrame()
    psi_dict={}
    for c in tqdm(f_cols):
        try:
            t_train = train[c].fillna(-998)
            t_test = test[c].fillna(-998)
            #获取切分点
            bins=[]
            for i in np.arange(0,1.1,0.2):
                bins.append(t_train.quantile(i))
            bins=sorted(set(bins))
            bins[0]=-np.inf
            bins[-1]=np.inf
            #计算psi
            t_psi = pd.DataFrame()
            t_psi['train'] = pd.cut(t_train,bins).value_counts().sort_index()
            t_psi['test'] = pd.cut(t_test,bins).value_counts()
            t_psi.index=[str(x) for x in t_psi.index]
            t_psi.loc['总计',:] = t_psi.sum()
            t_psi['train_rate'] = t_psi['train']/t_psi.loc['总计','train']
            t_psi['test_rate'] = t_psi['test']/t_psi.loc['总计','test']
            t_psi['psi'] = (t_psi['test_rate']-t_psi['train_rate'])*(np.log(t_psi['test_rate'])-np.log(t_psi['train_rate']))
            t_psi.loc['总计','psi'] = t_psi['psi'].sum()
            t_psi.index.name=c
            #汇总
            t_res = pd.DataFrame([[c,t_psi.loc['总计','psi']]],
                                 columns=['变量名','PSI'])
            psi_res = pd.concat([psi_res,t_res])
            psi_dict[c]=t_psi
            print(c,'done')
        except:
            print(c,'error')
    return psi_res,psi_dict

In [3]:
def correlation(df, threshold=0.98):
    """
    特征相关性计算
    @param df: 
    @param threshold: 
    @return: 
    """
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colName = corr_matrix.columns[i]
                col_corr.add(colName)
    return col_corr

In [4]:
train_df = pd.read_hdf('../input/train.h5')
test_df = pd.read_hdf('../input/test.h5')
sub = pd.DataFrame(test_df['time'])

In [5]:
train_df = train_df[train_df['temperature'].notnull()]
train_df = train_df.fillna(method='bfill')
test_df = test_df.fillna(method='bfill')

In [6]:
train_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                    'indoorHum', 'indoorAtmo', 'temperature']
test_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                   'indoorHum', 'indoorAtmo']

In [7]:
print('train_df.shape: ', train_df.shape)
train_df = train_df.loc[(train_df['outdoorTemp'] >= test_df['outdoorTemp'].min()) & (train_df['outdoorTemp'] <= test_df['outdoorTemp'].max())]
print('处理后 train_df.shape: ', train_df.shape)

train_df.shape:  (24807, 13)
处理后 train_df.shape:  (19338, 13)


In [8]:
data_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [9]:
numerical_features = ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']
diff_features = ['{}_diff'.format(i) for i in numerical_features]
numerical_diff_features = numerical_features + diff_features

In [10]:
for i in tqdm(numerical_features):
    data_df['{}_diff'.format(i)] = data_df[i].diff()

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1253.98it/s]


In [11]:
data_df.fillna(method='bfill', inplace=True)

In [12]:
# 基本聚合特征
group_feats = []
for f in tqdm(numerical_diff_features):
    data_df['MDH_{}_medi'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('median')
    data_df['MDH_{}_mean'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('mean')
    data_df['MDH_{}_max'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('max')
    data_df['MDH_{}_min'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('min')
    data_df['MDH_{}_sum'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('sum')
    data_df['MDH_{}_std'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('std')
    data_df['MDH_{}_skew'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('skew')
    
    data_df['MD_{}_medi'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('median')
    data_df['MD_{}_mean'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('mean')
    data_df['MD_{}_max'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('max')
    data_df['MD_{}_min'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('min')
    data_df['MD_{}_sum'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('sum')
    data_df['MD_{}_std'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('std')
    data_df['MD_{}_skew'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('skew')

    group_feats.append('MDH_{}_medi'.format(f))
    group_feats.append('MDH_{}_mean'.format(f))
    group_feats.append('MDH_{}_max'.format(f))
    group_feats.append('MDH_{}_min'.format(f))
#     group_feats.append('MDH_{}_sum'.format(f))
#     group_feats.append('MDH_{}_std'.format(f))
#     group_feats.append('MDH_{}_skew'.format(f))
    
    group_feats.append('MD_{}_medi'.format(f))
    group_feats.append('MD_{}_mean'.format(f))
    group_feats.append('MD_{}_max'.format(f))
    group_feats.append('MD_{}_min'.format(f))
#     group_feats.append('MD_{}_sum'.format(f))
#     group_feats.append('MD_{}_std'.format(f))
#     group_feats.append('MD_{}_skew'.format(f))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.87it/s]


In [13]:
# 基本交叉特征
for f1 in tqdm(numerical_diff_features + group_feats):
    for f2 in numerical_diff_features + group_feats:
        if f1 != f2:
            colname_substract = '{}_{}_subtract'.format(f1, f2)
            colname_add = '{}_{}_add'.format(f1, f2)            
            colname_multiply = '{}_{}_multyply'.format(f1, f2)
            colname_ratio = '{}_{}_ratio'.format(f1, f2)
            
            data_df[colname_substract] = data_df[f1].values - data_df[f2].values
            data_df[colname_add] = data_df[f1].values + data_df[f2].values
            data_df[colname_multiply] = data_df[f1].values / (data_df[f2].values + 0.001)
            data_df[colname_ratio] = data_df[f1].values * data_df[f2].values

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [15:52<00:00, 10.58s/it]


In [14]:
data_df = data_df.fillna(method='bfill')

In [15]:
# 历史信息提取
data_df['dt'] = data_df['day'].values + (data_df['month'].values - 3) * 31

for f in (numerical_diff_features + ['temperature']):
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        tmp = data_df.loc[data_df['dt'] < t, :].groupby(['hour'], as_index=False)[f].agg({
            'hit_{}_mean'.format(f): 'mean',
            'hit_{}_median'.format(f): 'median',
            'hit_{}_max'.format(f): 'max',
            'hit_{}_min'.format(f): 'min',
            'hit_{}_sum'.format(f): 'sum',
            'hit_{}_std'.format(f): 'std',
            'hit_{}_skew'.format(f): 'skew'
        })
        tmp['dt'] = t
        tmp_df = tmp_df.append(tmp)
        del tmp
        gc.collect()

    
for f in (numerical_diff_features + ['temperature']):
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        tmp = data_df.loc[(data_df['dt'] < t) & (data_df['dt'] >= t - 1), :].groupby(['hour'], as_index=False)[f].agg({
            'hit_t_1_{}_mean'.format(f): 'mean',
            'hit_t_1_{}_median'.format(f): 'median',
            'hit_t_1_{}_max'.format(f): 'max',
            'hit_t_1_{}_min'.format(f): 'min',
            'hit_t_1_{}_sum'.format(f): 'sum',
            'hit_t_1_{}_std'.format(f): 'std',
            'hit_t_1_{}_skew'.format(f): 'skew'
        })
        tmp['dt'] = t
        tmp_df = tmp_df.append(tmp)
        del tmp
        gc.collect()

    data_df = data_df.merge(tmp_df, on=['dt', 'hour'], how='left')
    del tmp_df
    gc.collect()

    
for f in (numerical_diff_features + ['temperature']):
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        tmp = data_df.loc[(data_df['dt'] < t) & (data_df['dt'] >= t - 2), :].groupby(['hour'], as_index=False)[f].agg({
            'hit_t_2_{}_mean'.format(f): 'mean',
            'hit_t_2_{}_median'.format(f): 'median',
            'hit_t_2_{}_max'.format(f): 'max',
            'hit_t_2_{}_min'.format(f): 'min',
            'hit_t_2_{}_sum'.format(f): 'sum',
            'hit_t_2_{}_std'.format(f): 'std',
            'hit_t_2_{}_skew'.format(f): 'skew'
        })
        tmp['dt'] = t
        tmp_df = tmp_df.append(tmp)
        del tmp
        gc.collect()

    data_df = data_df.merge(tmp_df, on=['dt', 'hour'], how='left')
    del tmp_df
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:15<00:00,  1.89it/s]


In [16]:
data_df = data_df.fillna(method='bfill')

In [17]:
# 离散化
for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
    data_df[f + '_20_bin'] = pd.cut(data_df[f], 20, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_50_bin'] = pd.cut(data_df[f], 50, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_100_bin'] = pd.cut(data_df[f], 100, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_200_bin'] = pd.cut(data_df[f], 200, duplicates='drop').apply(lambda x: x.left).astype(int)

In [18]:
for f1 in tqdm(['outdoorTemp_20_bin', 'outdoorHum_20_bin', 'outdoorAtmo_20_bin', 'indoorHum_20_bin', 'indoorAtmo_20_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')
        data_df['{}_{}_sum'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('sum')
        data_df['{}_{}_std'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('std')
        data_df['{}_{}_skew'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('skew')

for f1 in tqdm(['outdoorTemp_50_bin', 'outdoorHum_50_bin', 'outdoorAtmo_50_bin', 'indoorHum_50_bin', 'indoorAtmo_50_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')
        data_df['{}_{}_sum'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('sum')
        data_df['{}_{}_std'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('std')
        data_df['{}_{}_skew'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('skew')

for f1 in tqdm(['outdoorTemp_100_bin', 'outdoorHum_100_bin', 'outdoorAtmo_100_bin', 'indoorHum_100_bin',
                'indoorAtmo_100_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')
        data_df['{}_{}_sum'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('sum')
        data_df['{}_{}_std'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('std')
        data_df['{}_{}_skew'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('skew')

for f1 in tqdm(['outdoorTemp_200_bin', 'outdoorHum_200_bin', 'outdoorAtmo_200_bin', 'indoorHum_200_bin',
                'indoorAtmo_200_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')
        data_df['{}_{}_sum'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('sum')
        data_df['{}_{}_std'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('std')
        data_df['{}_{}_skew'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('skew')

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [24:30<00:00, 294.11s/it]


In [19]:
train_count = train_df.shape[0]
train_df = data_df[:train_count].copy().reset_index(drop=True)
test_df = data_df[train_count:].copy().reset_index(drop=True)
del data_df
gc.collect()

29

In [20]:
drop_columns = ["time", "year", "sec", "temperature"]

features = train_df[:1].drop(drop_columns, axis=1).columns
x_train = train_df[features]
x_test = test_df[features]

In [21]:
y_train = train_df['temperature'].values - train_df['outdoorTemp'].values

In [22]:
psi_res, psi_dict = get_psi(x_train, x_test, features)

  0%|                                                                                | 7/33069 [00:00<19:58, 27.58it/s]

month done
day done
hour done
min done
outdoorTemp done
outdoorHum done
outdoorAtmo done
indoorHum done


  0%|                                                                               | 16/33069 [00:00<16:41, 33.00it/s]

indoorAtmo done
outdoorTemp_diff done
outdoorHum_diff done
outdoorAtmo_diff done
indoorHum_diff done
indoorAtmo_diff done
MDH_outdoorTemp_medi done
MDH_outdoorTemp_mean done
MDH_outdoorTemp_max done


  0%|                                                                               | 24/33069 [00:00<15:38, 35.20it/s]

...

indoorAtmo_200_bin_outdoorHum_std done
indoorAtmo_200_bin_outdoorHum_skew done
indoorAtmo_200_bin_outdoorAtmo_medi done
indoorAtmo_200_bin_outdoorAtmo_mean done
indoorAtmo_200_bin_outdoorAtmo_max done
indoorAtmo_200_bin_outdoorAtmo_min done
indoorAtmo_200_bin_outdoorAtmo_sum done
indoorAtmo_200_bin_outdoorAtmo_std done


100%|███████████████████████████████████████████████████████████████████████████▉| 33062/33069 [14:53<00:00, 35.90it/s]

indoorAtmo_200_bin_outdoorAtmo_skew done
indoorAtmo_200_bin_indoorHum_medi done
indoorAtmo_200_bin_indoorHum_mean done
indoorAtmo_200_bin_indoorHum_max done
indoorAtmo_200_bin_indoorHum_min done
indoorAtmo_200_bin_indoorHum_sum done
indoorAtmo_200_bin_indoorHum_std done
indoorAtmo_200_bin_indoorHum_skew done


100%|████████████████████████████████████████████████████████████████████████████| 33069/33069 [14:53<00:00, 37.00it/s]

indoorAtmo_200_bin_indoorAtmo_medi done
indoorAtmo_200_bin_indoorAtmo_mean done
indoorAtmo_200_bin_indoorAtmo_max done
indoorAtmo_200_bin_indoorAtmo_min done
indoorAtmo_200_bin_indoorAtmo_sum done
indoorAtmo_200_bin_indoorAtmo_std done
indoorAtmo_200_bin_indoorAtmo_skew done


In [23]:
features = list(psi_res[psi_res['PSI'] <= 0.2]['变量名'].values) + ['outdoorTemp']

In [24]:
x_train = x_train[features]
x_test = x_test[features]

In [25]:
col_corr = correlation(x_train, 0.98)
print(col_corr)

{'MD_outdoorTemp_max_MDH_indoorHum_min_add', 'indoorAtmo_diff_MDH_indoorAtmo_mean_subtract', 'MD_outdoorTemp_diff_medi_MDH_indoorAtmo_mean_subtract', 'MD_indoorHum_diff_max_MDH_indoorHum_min_subtract', 'outdoorHum_MD_outdoorTemp_diff_max_add', 'MDH_outdoorHum_diff_medi_MDH_outdoorHum_diff_min_subtract', 'MDH_outdoorHum_min_MD_indoorAtmo_max_subtract', 'MD_outdoorHum_mean_MDH_outdoorTemp_min_add', 'MDH_outdoorAtmo_mean_MD_indoorHum_min_multyply', 'MD_indoorHum_mean_MD_indoorHum_diff_mean_add', 'MDH_outdoorHum_max_MD_outdoorHum_diff_mean_subtract', 'MDH_indoorAtmo_diff_mean_MDH_indoorHum_min_subtract', 'MDH_outdoorHum_medi_MD_outdoorHum_max_multyply', 'MDH_outdoorTemp_diff_medi_MDH_indoorHum_max_subtract', 'MD_outdoorAtmo_mean_MDH_indoorHum_max_subtract', 'MDH_outdoorTemp_diff_mean_MD_outdoorHum_max_add', 'MD_outdoorTemp_mean_MDH_outdoorTemp_max_subtract', 'MD_indoorHum_mean_MDH_indoorHum_mean_subtract', 'MDH_outdoorAtmo_medi_MDH_indoorHum_medi_multyply', 'MDH_outdoorHum_max_MD_indoorHum

In [26]:
x_train.drop(list(col_corr), axis=1, inplace=True)
x_test.drop(list(col_corr), axis=1, inplace=True)

In [27]:
nums = int(x_train.shape[0] * 0.8)

trn_x, trn_y, val_x, val_y = x_train[:nums], y_train[:nums], x_train[nums:], y_train[nums:]

train_matrix = xgb.DMatrix(trn_x, label=trn_y, missing=np.nan)
valid_matrix = xgb.DMatrix(val_x, label=val_y, missing=np.nan)
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

params = {
    'booster': 'gbtree',
    'eval_metric': 'mae',
    'min_child_weight': 5,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'eta': 0.01,
    'seed': 2020,
    'nthread': 36,
    'silent': 1
}

watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]

model_eval = xgb.train(params,
                       train_matrix,
                       num_boost_round=50000,
                       evals=watchlist,
                       verbose_eval=500,
                       early_stopping_rounds=1000)
val_pred = model_eval.predict(valid_matrix, ntree_limit=model_eval.best_ntree_limit).reshape(-1, 1)

[0]	train-mae:0.72618	eval-mae:0.31634
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 1000 rounds.
[500]	train-mae:0.05208	eval-mae:0.15542
[1000]	train-mae:0.04149	eval-mae:0.15390
[1500]	train-mae:0.03484	eval-mae:0.15327
[2000]	train-mae:0.02950	eval-mae:0.15324
[2500]	train-mae:0.02557	eval-mae:0.15298
[3000]	train-mae:0.02237	eval-mae:0.15294
[3500]	train-mae:0.01980	eval-mae:0.15291
[4000]	train-mae:0.01761	eval-mae:0.15285
[4500]	train-mae:0.01582	eval-mae:0.15291
[5000]	train-mae:0.01436	eval-mae:0.15282
[5500]	train-mae:0.01310	eval-mae:0.15282
[6000]	train-mae:0.01202	eval-mae:0.15277
[6500]	train-mae:0.01109	eval-mae:0.15274
Stopping. Best iteration:
[5744]	train-mae:0.01255	eval-mae:0.15272



In [28]:
mse = mean_squared_error(val_y, val_pred)
print("mse_score:", mse)
print("mse_score:", str(mse)[2:7])

mse_score: 0.03919283827443374
mse_score: 03919


In [29]:
feat_imp_dict = model_eval.get_score(importance_type='gain')

In [30]:
feat_imp = pd.Series(feat_imp_dict).sort_values(ascending=False).reset_index().rename(columns={'index': 'feature',
                                                                                               0: 'importance'})
feat_imp['normalized_importance'] = feat_imp['importance'] / feat_imp['importance'].sum()
feat_imp['cumulative_importance'] = np.cumsum(feat_imp['normalized_importance'])
record_low_importance = feat_imp[feat_imp['cumulative_importance'] > 0.95]

to_drop = list(record_low_importance['feature'])
print(to_drop)

['indoorAtmo_100_bin_outdoorAtmo_std', 'hour', 'MDH_outdoorTemp_diff_medi_MDH_indoorHum_diff_min_ratio', 'hit_t_1_outdoorAtmo_diff_sum', 'outdoorTemp_200_bin_outdoorTemp_skew', 'indoorHum_20_bin_outdoorAtmo_mean', 'MD_outdoorTemp_min_MDH_indoorAtmo_mean_add', 'MD_outdoorHum_max_MDH_indoorAtmo_mean_subtract', 'outdoorTemp_diff_MDH_outdoorHum_diff_min_ratio', 'MDH_outdoorHum_diff_mean_MDH_outdoorTemp_diff_mean_multyply', 'MD_outdoorHum_min_MD_outdoorHum_max_multyply', 'MDH_indoorAtmo_diff_sum', 'MD_outdoorHum_max_MDH_outdoorHum_diff_max_add', 'outdoorTemp_diff_MDH_outdoorHum_diff_max_ratio', 'outdoorTemp_200_bin_outdoorTemp_std', 'indoorAtmo_100_bin_outdoorHum_min', 'MD_outdoorAtmo_mean_MDH_indoorAtmo_max_subtract', 'outdoorTemp_MD_outdoorAtmo_medi_add', 'MD_indoorHum_max_MDH_indoorHum_max_multyply', 'outdoorHum_MD_indoorHum_diff_max_multyply', 'MDH_outdoorTemp_diff_medi_MD_outdoorAtmo_diff_mean_ratio', 'outdoorTemp_100_bin_outdoorTemp_std', 'MD_indoorAtmo_diff_mean_MD_outdoorTemp_diff_m

In [31]:
x_train.drop(to_drop, axis=1, inplace=True)
x_test.drop(to_drop, axis=1, inplace=True)

In [32]:
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

model = xgb.train(params,
                  train_all_matrix,
                  num_boost_round=model_eval.best_ntree_limit + 20)

test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)

In [33]:
sub['temperature'] = test_pred[:, 0] + test_df['outdoorTemp'].values
sub.to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), str(mse)[2:7]), index=False)